In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:
import os 
import glob
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from glob import glob

class TimeSeriesDataset(Dataset):
    def __init__(self, folder_nolayoff, folder_layoff, src_col):
        
        # csvs_layoffs = glob.glob(folder_layoff + "/*.csv")
        # csvs_nolayoffs = glob.glob(folder_nolayoff + "/*.csv")
        X_train = []
        Y_train = []
        all_csv_files = []
        EXT = "*.csv"  # Define the variable EXT
        for path, subdir, files in os.walk(folder_layoff):
            
            for file in glob(os.path.join(path, EXT)):
                # print(file)
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]
                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(1))
                # all_csv_files.append(file)

        for path, subdir, files in os.walk(folder_nolayoff):
            for file in glob(os.path.join(path, EXT)):
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]

                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(0))
                # all_csv_files.append(file)





        # for directory in csvs_layoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(1)
        #     # df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #     # X_train.append(df[src_col].values)
        #     # Y_train.append(1)
        # for directory in csvs_nolayoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(0)
            
        X_train = torch.from_numpy(np.array(X_train))
        Y_train = torch.from_numpy(np.array(Y_train))
        # if len(X_train.shape) < 3:
        #     X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape[1], X_train.shape[2])) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        self.X_train = X_train
        self.Y_train = Y_train
        self.num_channels = X_train.shape[1]
        self.len = X_train.shape[0]

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X_train[idx].float(), self.Y_train[idx]



In [3]:
df = pd.read_csv("stocks_layoffs/Energy/ENPH0.csv")

In [4]:
rolling_min = list(df['Open'].rolling(window=90).min())[-1]
rolling_max = list(df['Open'].rolling(window=90).max())[-1]

# Calculate the scaled prices
df['Scaled_Price_MinMax'] = (df["Open"] - rolling_min) / (rolling_max - rolling_min)

In [5]:
# avg = list(df['Open'].rolling(window=90).mean())[-1]
# df['Normalized_Price_minmax'] = df['Open'] / avg

In [6]:
# df["Normalized_Price_minmax"]

In [7]:
dataset_minmax = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "Scaled_Price_MinMax")
dataset_oipa = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_inproportion_to_average")
dataset_on = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_normalized")
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset_minmax)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SequentialSampler(val_indices)

train_loader_minmax = torch.utils.data.DataLoader(dataset_minmax, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size, 
                                           sampler=train_sampler)

validation_loader_minmax = torch.utils.data.DataLoader(dataset_minmax, batch_size=batch_size, sampler=valid_sampler)
validation_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, sampler=valid_sampler)
validation_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size,   sampler=valid_sampler)

In [8]:
from torch.utils.data import DataLoader

In [9]:
train_loader_minmax.dataset.Y_train

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [10]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model_minmax = BinaryClassifier()
model_open_inproportion_to_average = BinaryClassifier()
model_open_normalized = BinaryClassifier()
# Define the loss function and optimizer
criterion_minmax = nn.BCELoss()
criterion_oipa = nn.BCELoss()
criterion_on = nn.BCELoss()
optimizer_minmax = optim.Adam(model_minmax.parameters())
optimizer_oipa = optim.Adam(model_open_inproportion_to_average.parameters())
optimizer_on = optim.Adam(model_open_normalized.parameters())

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    ## Run through dataloader batches
    



    for i, (X_train, y_train) in enumerate(train_loader_minmax):
    
        # Forward pass
        # print(X_train)
        outputs_minmax = model_minmax(X_train)
        y_train = y_train.unsqueeze(1)
        loss_minmax = criterion_minmax(outputs_minmax.float(), y_train.float())
        optimizer_minmax.zero_grad()
        loss_minmax.backward()
        optimizer_minmax.step()
    for i, (X_train, y_train) in enumerate(train_loader_oipa):
        # Forward pass
        outputs_open_inproportion_to_average = model_open_inproportion_to_average(X_train)
        y_train = y_train.unsqueeze(1)
        loss_oipa = criterion_oipa(outputs_open_inproportion_to_average.float(), y_train.float())
        optimizer_oipa.zero_grad()
        loss_oipa.backward()
        optimizer_oipa.step()
    for i, (X_train, y_train) in enumerate(train_loader_on):
        # Forward pass
        outputs_open_normalized = model_open_normalized(X_train)
        y_train = y_train.unsqueeze(1)
        loss_on = criterion_on(outputs_open_normalized.float(), y_train.float())
        # Backward pass and optimization
        optimizer_on.zero_grad()
        loss_on.backward()
        optimizer_on.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss_minmax: {loss_minmax.item():.4f}, Loss_oipa: {loss_oipa.item():.4f}, Loss_on: {loss_on.item():.4f}')
    

# # Evaluation
# with torch.no_grad():
#     outputs = model(X_test)
#     predicted = (outputs >= 0.5).float()
#     accuracy = (predicted == y_test).float().mean()
#     print(f'Test Accuracy: {accuracy:.4f}')

# # Make predictions
# with torch.no_grad():
#     outputs = model(X_new)
#     predicted = (outputs >= 0.5).float()

c:\Users\alexs\anaconda3\envs\cs828\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/50], Loss_minmax: 0.6517, Loss_oipa: 0.6838, Loss_on: 0.8064
Epoch [2/50], Loss_minmax: 0.7514, Loss_oipa: 0.7585, Loss_on: 0.3687
Epoch [3/50], Loss_minmax: 0.7283, Loss_oipa: 0.5672, Loss_on: 0.9868
Epoch [4/50], Loss_minmax: 0.6424, Loss_oipa: 0.7276, Loss_on: 0.5601
Epoch [5/50], Loss_minmax: 0.8659, Loss_oipa: 0.5075, Loss_on: 0.7166
Epoch [6/50], Loss_minmax: 0.5788, Loss_oipa: 0.2813, Loss_on: 0.6219
Epoch [7/50], Loss_minmax: 0.6663, Loss_oipa: 0.2739, Loss_on: 0.7479
Epoch [8/50], Loss_minmax: 0.6101, Loss_oipa: 0.8635, Loss_on: 0.3153
Epoch [9/50], Loss_minmax: 0.8128, Loss_oipa: 0.2537, Loss_on: 0.6387
Epoch [10/50], Loss_minmax: 0.6501, Loss_oipa: 0.3386, Loss_on: 0.1208
Epoch [11/50], Loss_minmax: 0.5572, Loss_oipa: 0.7888, Loss_on: 0.6261
Epoch [12/50], Loss_minmax: 0.3159, Loss_oipa: 0.5283, Loss_on: 0.2896
Epoch [13/50], Loss_minmax: 0.8448, Loss_oipa: 0.8747, Loss_on: 0.2700
Epoch [14/50], Loss_minmax: 0.9982, Loss_oipa: 0.6149, Loss_on: 0.2547
Epoch [15/50], 

In [11]:
# Single model eval
# is_correct = []
# for i, (X_train, y_train) in enumerate(validation_loader_minmax):
    

#         # print(X_train)
#         # print(y_train)
#         # Forward pass
#     outputs = model(X_train)
#     out = outputs.squeeze(1)
#     output_bin = (out >= 0.5).int().item()
#     correct = output_bin == int(y_train.item())
#     is_correct.append(correct)
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")


In [12]:
# Majority Voting Ensemble Eval
is_correct_confidence = []
is_correct_vote = []
is_correct_minmax = []
is_correct_oipa = []
is_correct_on = []

for i, (X_train, y_train) in enumerate(validation_loader_minmax):
        # print(X_train)
        # print(y_train)
        # Forward pass
    X_train_minmax = validation_loader_minmax.dataset.X_train[i].unsqueeze(0).float()
    X_train_oipa = validation_loader_oipa.dataset.X_train[i].unsqueeze(0).float()
    X_train_on = validation_loader_on.dataset.X_train[i].unsqueeze(0).float()
    Y_train_minmax = validation_loader_minmax.dataset.Y_train[i]
    Y_train_oipa = validation_loader_oipa.dataset.Y_train[i]
    Y_train_on = validation_loader_on.dataset.Y_train[i]
    # Sanity check
    assert (Y_train_minmax.item() == Y_train_oipa.item())
    assert (Y_train_minmax.item() == Y_train_on.item())

    
    # print(outputs_minmax)
    outputs_minmax = model_minmax(X_train_minmax).squeeze(1)
    outputs_oipa = model_open_inproportion_to_average(X_train_oipa).squeeze(1)
    outputs_on = model_open_normalized(X_train_on).squeeze(1)

    ## Ensemble confidence based on average of outputs
    confidence_avg_out = (outputs_minmax + outputs_oipa + outputs_on) / 3
    confidence_output_bin = (confidence_avg_out >= 0.5).int().item()

    vote_minmax = (outputs_minmax >= 0.5).int().item()
    is_correct_minmax.append(vote_minmax == int(y_train.item()))
    vote_oipa = (outputs_oipa >= 0.5).int().item()
    is_correct_oipa.append(vote_oipa == int(y_train.item()))
    vote_on = (outputs_on >= 0.5).int().item()
    is_correct_on.append(vote_on == int(y_train.item()))
    ## Ensemble voting based on majority voting
    vote = (vote_minmax + vote_oipa + vote_on) >= 2
    correct_confidence = confidence_output_bin == int(y_train.item())
    correct_vote = vote == int(y_train.item())
    is_correct_confidence.append(correct_confidence)
    is_correct_vote.append(correct_vote)

    # out = outputs.squeeze(1)
    # output_bin = (out >= 0.5).int().item()
    # correct = output_bin == int(y_train.item())
    # is_correct.append(correct)
# Count ratio of trues and falses in is_correct
true_count_confidence = sum(is_correct_confidence)
false_count_confidence = len(is_correct_confidence) - true_count_confidence
print(f"True count (Confidence): {true_count_confidence}")
print(f"False count (Confidence): {false_count_confidence}")
print(f"True ratio (Confidence): {true_count_confidence / len(is_correct_confidence)}")
true_count_vote = sum(is_correct_vote)
false_count_vote = len(is_correct_vote) - true_count_vote
print(f"True count (Vote): {true_count_vote}")
print(f"False count (Vote): {false_count_vote}")
print(f"True ratio (Vote): {true_count_vote / len(is_correct_vote)}")
true_count_minmax = sum(is_correct_minmax)
false_count_minmax = len(is_correct_minmax) - true_count_minmax
print(f"True count (minmax): {true_count_minmax}")
print(f"False count (minmax): {false_count_minmax}")
print(f"True ratio (minmax): {true_count_minmax / len(is_correct_minmax)}")
true_count_oipa = sum(is_correct_oipa)
false_count_oipa = len(is_correct_oipa) - true_count_oipa
print(f"True count (OIPA): {true_count_oipa}")
print(f"False count (OIPA): {false_count_oipa}")
print(f"True ratio (OIPA): {true_count_oipa / len(is_correct_oipa)}")
true_count_on = sum(is_correct_on)
false_count_on = len(is_correct_on) - true_count_on
print(f"True count (ON): {true_count_on}")
print(f"False count (ON): {false_count_on}")
print(f"True ratio (ON): {true_count_on / len(is_correct_on)}")


True count (Confidence): 92
False count (Confidence): 11
True ratio (Confidence): 0.8932038834951457
True count (Vote): 92
False count (Vote): 11
True ratio (Vote): 0.8932038834951457
True count (minmax): 77
False count (minmax): 26
True ratio (minmax): 0.7475728155339806
True count (OIPA): 95
False count (OIPA): 8
True ratio (OIPA): 0.9223300970873787
True count (ON): 90
False count (ON): 13
True ratio (ON): 0.8737864077669902


In [13]:
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")

In [14]:
# Save each model
directory = "classifier_models/"
if not os.path.exists(directory):
    os.makedirs(directory)
torch.save(model_minmax.state_dict(), directory +"minmax_model.pth")
torch.save(model_open_inproportion_to_average.state_dict(), directory +"open_inproportion_to_average_model.pth")
torch.save(model_open_normalized.state_dict(), directory +"open_normalized_model.pth")


In [15]:
# Load in saved models
model_minmax = BinaryClassifier()
model_open_inproportion_to_average = BinaryClassifier()
model_open_normalized = BinaryClassifier()
model_minmax.load_state_dict(torch.load(directory +"minmax_model.pth"))
model_open_inproportion_to_average.load_state_dict(torch.load(directory +"open_inproportion_to_average_model_2.pth"))
model_open_normalized.load_state_dict(torch.load(directory +"open_normalized_model_2.pth"))


FileNotFoundError: [Errno 2] No such file or directory: 'classifier_models/open_inproportion_to_average_model_2.pth'

#